En este notebook, empezaremos a poner a Julia en acción. Los ejercicios de este notebook se harán en clase, aunque son el contenido de la tarea 1.

# El método de Newton

Como bien sabemos, el método de Newton es un método iterativo para encontrar los ceros, o raíces, de la ecuación $f(x)=0$. 
Partiendo de una aproximación $x_0$, que debe ser lo suficientemente cercana, y denotando la derivada de $f(x)$ como $f'(x)$, tenemos que los iterados vienen dados por:

$$
x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)} .
$$


### Ejercicio 1

- Escriban una función `newton` que, a partir de `f`, `fprime` y `x0` dados, obtenga una de las raices de la ecuación. Comprueben que funciona con $f(x)=x^2 -2$ y $f'(x) = 2x$. Tengan suficiente cuidado para que no haya ningún tipo de inestabilidad de tipo en su función.

- Documenta la función de manera adecuada (*docstrings*).

- ¿Cómo se comporta, en términos del número de iterados, la convergencia del método de Newton?

#### *Definición y documentación de la función*:

Para implementar el método unidimensional, se puede hacer lo siguiente:

In [ ]:
"""
    newton(f, fprime, x0, número_iteraciones)

`newton` es una implementación unidimensional real del método de Newton para encontrar raíces. 

# Argumentos

Para poder utilizar la función se requieren los siguientes argumentos:

* `f`, la función real de variable real de la que se quiere buscar una raíz (un punto ``a`` tales que ``f(a) = 0``),
* `f_prime`, la función derivada de `f`, y,
* `x0`, una adivinanza inicial sobre la posición de la raíz.

Opcionalmente, se puede especificar lo siguiente:

* `número_iteraciones`, el número de iteraciones a realizar. (Por defecto está configurado en 10000 iteraciones.)

`newton` requiere que tanto `f` como `fprime` sean funciones, que `x0` sea un real y que `número_iteraciones` sea un número entero. `x0`, en particular, siempre es convertida a un número flotante para mejorar la estabilidad de tipo.

# Ejemplos
```julia-repl
    julia > newton(x -> x^2 - 2, x -> 2*x, 2, 5)
    1.4142135623730951

    julia > newton(x -> x^2 - 2, x -> 2*x, 1.4, 100)
    1.414213562373095

    julia > newton(x -> x^2 - 2, x -> 2*x, φ, 1)
    1.4270509831248424

    julia > newton(x -> x^2 - 2, x -> 2*x, φ, 10)
    1.414213562373095    
```
"""
function newton(f::Function, fprime::Function, x0::Real, número_iteraciones::Int = 10000)
    
    punto_actual = float(x0)
    
    for i in 1:número_iteraciones
        punto_actual -= f(punto_actual)/fprime(punto_actual)
    end
    
    return(punto_actual)
end

#### *Ejemplo del cálculo de √2*:

In [ ]:
#Generando el ejemplo con los siguientes parámetros:

f(x) = x^2 - 2
g(x) = 2*x
x0 = 1.5
iteraciones = 10

#La raíz cuadrada positiva de 2 calculada con el método es:

@show raíz_newton = newton(f, g, x0, iteraciones)

#La raíz cuadrada (positiva) de 2 es:

@show raíz = sqrt(2)

#Comparando si son iguales como números flotantes de 64 bits:

@show raíz_newton == raíz

Por lo que el método implementado calculó correctamente la raíz cuadrada positiva de 2 como número flotante de 64 bits con 10 iteraciones, con la adivinanza inicial igual a 1.5.

#### *Análisis de la estabilidad de tipo*:

Para ver la estabilidad de tipo en un ejemplo, basta con usar el macro `@code_warntype` en la llamada a la función:

In [ ]:
#Para el análisis se consideran las funciones y el número de iteraciones definidas para el ejemplo anterior. Lo que se va a modificar va a ser el tipo de adivinanza inicial suministrada.

x0 = 1.5 #Lo mismo que en el ejemplo anterior. 
@show typeof(x0) #Número flotante de 64 bits.

@code_warntype newton(f, g, x0, iteraciones)

In [ ]:
#El macro no emitió advertencias. Intentando con un entero:

x0 = 2
@show typeof(x0) #Número entero de 64 bits.

@code_warntype newton(f, g, x0, iteraciones)

In [ ]:
#El macro tampoco emitió advertencias. Intentando con un irracional:

x0 = φ #El número áureo
@show typeof(x0) #Número irracional

@code_warntype newton(f, g, x0, iteraciones)

In [ ]:
#Tampoco emitió advertencias, esto debido al comportamiento de un irracional ante la función float(): se toma la representación como número flotante de 64 bits más cercana a dicho irracional. 

float(φ)

Estos ejemplos otorgan evidencia sobre la estabilidad de tipo de la función implementada. En todos los ejemplos se devolvió un número flotante de 64 bits.

#### Análisis de la convergencia:

Para analizar la convergencia del método, consideremos el ejemplo en el que se calcula la raíz cuadrada de dos con una adivinanza inicial constante. ¿Cómo cambia la diferencia entre el valor dado por Julia y el valor calculado según el número de iteraciones?

Para hacer esto de forma automática y así poder graficar, conviene definir:

* Una función que otorgue la diferencia entre el valor de la raíz cuadrada de un número $a$ calculada por el método de Newton implementado y el valor de la misma raíz cuadrada calculada por Julia con una adivinanza inicial fija y un número de iteraciones dado.
    
* Un conjunto de valores de esta diferencia para diferentes cantidades de iteraciones.

In [ ]:
"""
    diferencia_raíz_cuadrada(a, x0, número_iteraciones)

`diferencia_raíz cuadrada` calcula la diferencia (signada) entre el valor de la raíz cuadrada positiva del número ``a`` calculada mediante la función `sqrt` implementada en Julia y el valor de la misma raíz cuadrada calculada mediante el método de Newton.

# Argumentos

Para poder utilizar la función se requieren los siguientes argumentos:

* `a`, el número del que se está calculando la raíz,
* `x0`, una adivinaza de la raíz (positiva) de ``a``, y, 
* `número_iteraciones`, el número de iteraciones realizadas en el método de Newton para calcular la raíz.

#Ejemplos:
```julia-repl
    julia> diferencia_raíz_cuadrada(2, 2, 1)
    -0.08578643762690485

    julia> diferencia_raíz_cuadrada(2, 2, 3)
    -2.1239014147411694e-6

    julia> diferencia_raíz_cuadrada(π, 1.5, 1)
    -0.02474370029108175

    julia> diferencia_raíz_cuadrada(π, 1.5, 3)
    -8.183899780078718e-9
```
"""
function diferencia_raíz_cuadrada(a::Real, x0::Number, número_iteraciones::Int)
    
    raíz_cuadrada_newton = newton(x -> x^2 - a, x -> 2*x, x0, número_iteraciones)
    raíz_julia = sqrt(a)
    
    diferencia = raíz_julia - raíz_cuadrada_newton
    
    return(diferencia)
end

### Ejercicio 2

- Usando la función que hicieron en el ejercicio anterior y variando la condición inicial `x0`, de -3 a 3 con pasos suficientemente pequeños, por ejemplo `0.125`, grafiquen la dependencia de la raíz encontrada de la condición inicial para $f(x)=x^2-2$. Para hacer esto, vale la pena que guarden en un vector la raíz obtenida y en otro la condición inicial.

- Repitan el inciso anterior para $g(x) = (x-1)(x-2)(x-3)$, considerando el intervalo $x_0\in[1,3]$ y muchas condiciones iniciales.

### Ejercicio 3

Considerando la función $h(z)=z^3-c$, con $z\in\mathbb{C}$ y $c=1$, utiliza el método de Newton para encontrar las raices en el plano complejo de dicha ecuación. Repite el ejercicio de la dependencia de la raíz encontrada de las condiciones iniciales, graficando en el plano de Argand.

---

# Derivación numérica

En principio sabes derivar. Sin embargo, por distintas razones a veces es necesario implementar la derivada numéricamente. A continuación implementarán distintos métodos para hacer esto.

La derivada, como bien sabemos, se define como:

$$
f'(x0) = \frac{{\rm d}f}{{\rm d}x}(x_0) \equiv \lim_{h\to 0} \frac{f(x_0+h)-f(x_0)}{h}.
$$

Sin embargo, esta definición es difícil de implementar por el límite $h\to0$.

### Ejercicio 4

- Definan una función `derivada_derecha` que calcule *numéricamente* la derivada de una función $f(x)$ de una variable (a priori arbitaria), en un punto $x_0$. Para esto, utilizaremos la aproximación de la derivada
que se basa en su definición:

$$ 
f'(x_0) \approx \frac{\Delta f_+}{\Delta x} \equiv \frac{f(x_0+h)-f(x_0)}{h},
$$

lo que requiere de la especificación de `h`. (Este método también se conoce por el nombre de *diferencias finitas*.)

- A fin de simular el $\lim_{h\to 0}$, consideren distintos valores de $h$ que precisamente simulen dicho límite. Para cada valor de $h$ calculen el error absoluto del cálculo numérico respecto al valor *exacto*. Ilustren esto en una gráfica del error vs $h$, para $f(x) = 3x^3-2$, en $x_0=1$. 

### Ejercicio 5

- Repitan el ejercicio anterior (escribe una función `derivada_simetrica`, usando ahora la aproximación *simétrica* (respecto a la ocurrencia de $h$) de la derivada, dada por

$$
f'(x_0) \approx \frac{ \Delta f_{sym}}{\Delta x} = \lim_{h\to 0} \frac{f(x_0+h)-f(x_0-h)}{2h}.
$$

- ¿Por qué es correcto afirmar que la derivada simétrica resulta en una mejor aproximación que la derivada derecha? Argumenten y si es necesario usen argumentos analíticos.

### Ejercicio 6

- Definan la función `derivada_compleja` considerando la definición de la derivada dada por:
$$
f'(x_0) \approx \frac{\Delta f_{cmplx}(x_0)}{\Delta x} = \Im\Big(\frac{f(x_0+ i h)}{h}\Big),
$$
donde $\Im$ indica la parte imaginaria del argumento ($i=\sqrt{-1}$). 

- Argumenta (analíticamente) por qué esta definición da tan buen resultado. ¿Hay algún caso en que esta definición puede dar resultados inesperados?